In [ ]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from beakerx import *
plt.style.use('ggplot')
import seaborn as sns
sns.set()
import h5py
import os
from tqdm import tqdm_notebook as tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

In [1]:
cd ..

/home/xupeng/projects/synapse


## read 15 and 48 section of A

In [10]:
crack_path = 'data/crack/'

In [11]:
ls data/crack/

9.7/              im015.png              im048_warp0_ip_db.png
align_0_15_1.hdf  im015_reverse0.png     im048_warp0.png
align_0_15_2.hdf  im015_warp0_ip_db.png  im048_warped1.png
align_0_48_1.hdf  im015_warp0.png        im048_warped2.png
align_0_48_2.hdf  im015_warped1.png      im049.png
align_1_15_1.hdf  im015_warped2.png      im15_prediction1.png
align_1_15_2.hdf  im016.png              im15_prediction2.png
align_1_48_1.hdf  im047.png              im48_prediction1.png
align_1_48_2.hdf  im048_p1.png           im48_prediction2.png
im014.png         im048_p2.png           im_A+_v2_200_nocrack.h5
im015_p1.png      im048.png
im015_p2.png      im048_reverse0.png


### read predicted A

In [18]:
pred_file_path = 'data/prediction/test_before_realignment/'
pred_A = 'im_A+_v2_200_pred.h5'
pred_B = 'im_B+_v2_200_pred.h5'
pred_C = 'im_C+_v2_200_pred.h5'

In [19]:
with h5py.File(pred_file_path+pred_A) as f:
    prediction_14 = f['main'][14]
    prediction_47 = f['main'][47]

In [20]:
def get_roi(image,anotherimage):
    img = np.copy(image)
    img_ = np.copy(image)
    labels = label(image)
    img[np.where(labels[0]==1)] =0
    img_[np.where(labels[0]==2)] =0
    anoimg = np.copy(anotherimage)
    anoimg_ = np.copy(anotherimage)
    anoimg[np.where(labels[0]==1)] =0
    anoimg_[np.where(labels[0]==2)] =0
    return anoimg,anoimg_

## process and save as png

In [29]:
crack_path+'tmp'

'data/crack/tmp'

In [36]:
if os.path.exists(crack_path+'tmp'):
    os.system('rm -rf '+crack_path+'tmp')
    os.mkdir(crack_path+'tmp')
else:
    os.mkdir(crack_path+'tmp')

In [15]:
scipy.misc.imsave(crack_path+'tmp/im15_prediction1.png',get_roi(scipy.misc.imread(crack_path+'im015_warp0.png'),prediction_14.T)[1])
scipy.misc.imsave(crack_path+'tmp/im15_prediction2.png',get_roi(scipy.misc.imread(crack_path+'im015_warp0.png'),prediction_14.T)[0])
scipy.misc.imsave(crack_path+'tmp/im48_prediction1.png',get_roi(scipy.misc.imread(crack_path+'im048_warp0.png'),prediction_47.T)[1])
scipy.misc.imsave(crack_path+'tmp/im48_prediction2.png',get_roi(scipy.misc.imread(crack_path+'im048_warp0.png'),prediction_47.T)[0])

## matlab script to deal with crack(translated to python)

In [91]:
ls data/crack/

9.7/              im015.png              im048_warp0_ip_db.png
align_0_15_1.hdf  im015_reverse0.png     im048_warp0.png
align_0_15_2.hdf  im015_warp0_ip_db.png  im048_warped1.png
align_0_48_1.hdf  im015_warp0.png        im048_warped2.png
align_0_48_2.hdf  im015_warped1.png      im049.png
align_1_15_1.hdf  im015_warped2.png      im15_prediction1.png
align_1_15_2.hdf  im016.png              im15_prediction2.png
align_1_48_1.hdf  im047.png              im48_prediction1.png
align_1_48_2.hdf  im048_p1.png           im48_prediction2.png
im014.png         im048_p2.png           im_A+_v2_200_nocrack.h5
im015_p1.png      im048.png              tmp/
im015_p2.png      im048_reverse0.png


In [93]:
ind = [15,48]
vid = 0
for i in ind:
    for j in np.arange(1,3):
        #print ('data/crack/im'+'{0:03}'.format(i)+'_warped'+'{0:01}'.format(j)+'.png')
        im = imread('data/crack/im'+'{0:03}'.format(i)+'_warped'+'{0:01}'.format(j)+'.png')
        sz = im.shape
        tmp = h5py.File('data/crack/align_'+str(vid)+'_'+str(i)+'_'+str(j)+'.hdf')['main'][:]
        tmpp = tmp[1]
        if j ==1:
            B = imwarp(im, invert(affine2d(tmpp)),'FillValues',0,'OutputView',imref2d(sz))
        else:
            B = B+imwarp(im, invert(affine2d(tmpp)),'FillValues',0,'OutputView',imref2d(sz))
    imwrite('data/crack/im'+'{0:03}'.format(i)+'_reversed'+'{0:01}'.format(vid)+'.png',B)   

## read png and replace original 15 and 48

In [ ]:
with h5py.File(pred_file_path+pred_A) as f:
    resultsA_reverse1 = f['main'][:]
resultsA_reverse1[14] = scipy.misc.imread(crack_path+'im015_reverse0.png').T
resultsA_reverse1[47] = scipy.misc.imread(crack_path+'im048_reverse0.png').T
with h5py.File(pred_file_path+'im_A+_v2_200_pred_add_crack.h5') as f:
    f.create_dataset('main',data=resultsA_reverse1)

## reverse all A, B, C using matlab script(translated  to python)

#### test if the script is right:
- pred_file_path

    - data/prediction/test_before_realignment
    - im_A+_v2_200_pred.h5,im_B+_v2_200_pred.h5,im_C+_v2_200_pred.h5
- reversed:
    - data/reverse
    - results_new_A+_v2_200.h5,results_new_B+_v2_200.h5,results_new_C+_v2_200.h5

In [38]:
volume_name = np.array(['A','B','C','A+','B+','C+'])
bad_slices = np.array([[143],[1,29,30,58,59,91],[28,88,100],
    [65,93,94,122,123,125],[1,29,30,58,59,91],[28,88,100]])

gg = np.array([[142],[0,28,31,57,60,90],[27,87,99],
    [64,92,95,121,124,126],[0,28,31,57,60,90],[27,87,99]])

new_align_sz = np.array([[1727,1842],[2069,1748],[1986,2036],
                         [1741,1912],[2898,1937],[1914,1983]]) 
def align2origin(prediction_path,reverse_path):
    nn=volume_name
    sz = new_align_sz
    for nid in np.arange(4,6):
        vol = nn[nid]
        print (vol)
        pw=0
        ph=0
        if vol =='B+':
            ph=700
        # load/crop result
        result = h5py.File(prediction_path+'/im_'+vol+'_v2_200_pred.h5')['main'][:]
        sz_r = result.shape
        print ('shape: ', sz_r)
        sz_bd = np.round((sz_r - np.array([125,sz[nid][0]-400,sz[nid][0]-400]))/2.).astype('int')
        print (sz_bd)
        result = result[sz_bd[0]:-sz_bd[0], sz_bd[1]:-sz_bd[1], sz_bd[2]:-sz_bd[2]]

        pp = np.cumsum(np.loadtxt('data/align/trans_'+vol+'_v2.txt',delimiter=','),axis=0)
        pp = pp[76] - pp
        ww = np.ceil([np.max(pp[:,0]),np.max(pp[:,1]),-np.min(pp[:,0]),-np.min(pp[:,1])]).astype('int')
        #pp=pp[14:-14,:]
        # 1250+200*2
        print ('result shape', result.shape)
        result_o = np.zeros([125,1250,1250],'uint16');
        for i in range(125):
            pd = np.round(pp[i+14-1,:]).astype('int') 
            tmp = np.zeros([3072,3072])
            tmp2 = np.pad(tmp,(ph,pw),'symmetric')
            print (tmp2.shape)
            print (pd,ww,ph)
            print ([(911+pd[0]-ww[0]+ph),(910+pd[0]+1250+ww[2]+ph),
                    (911+pd[1]-ww[1]+pw),(910+pd[1]+1250+ww[3]+pw)])
            print ([(910+pd[0]+1250+ww[2]+ph)-(911+pd[0]-ww[0]+ph),
                    (910+pd[1]+1250+ww[3]+pw)-(911+pd[1]-ww[1]+pw)])
            tmp2[(911+pd[0]-ww[0]+ph):(910+pd[0]+1250+ww[2]+ph),
                    (911+pd[1]-ww[1]+pw):(910+pd[1]+1250+ww[3]+pw)] = result[i,:,:]
            result_o[i,:,:] = tmp2[ph+911:ph+910+1250,pw+911:pw+910+1250]
        with h5py.File(reverse_path+'results_new_'+vol+'.h5') as f:
            f.create_dataset('main',data = result_o)
        print ('save reversed result: '+vol)

In [ ]:
align2origin(prediction_path='data/prediction/test_before_realignment/',reverse_path='data/reverse/9.10_test/')